In [1]:
import os
import numpy as np
import tensorflow as tf
import pickle as pkl
import json
from mqtt_sub import sub_iot_platform_cin_con
from MobiusAPI import http_post_get

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
# 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[3],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8000)])
    except RuntimeError as e:
        # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

2023-10-17 05:41:53.278965: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 05:41:53.422623: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-17 05:41:53.458053: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-17 05:41:54.156923: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
model = tf.keras.models.load_model('model/resnet_disk')

nu = "mqtt://203.250.148.120/sensor1RawDataSub?ct=json"
Port = 20516

URI = '/Mobius/radarEye/report/countingDiskType'
AE_ID = 'radarEye1'

2023-10-17 05:41:55.266277: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 05:41:55.924453: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8000 MB memory:  -> device: 3, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d8:00.0, compute capability: 8.6


In [3]:
while True:
    json_data = sub_iot_platform_cin_con(nu, Port)
    print(type(json_data['data_1']))
    data = np.array(json_data['data_1'])
    print(data.shape)
    y_pred = model.predict(data)
    # print(y_pred)
    y = y_pred[0]
    pred = dict()
    for i in range(len(y)):
        pred[str(i).zfill(2)] = y[i].tolist()

    send_data = json.dumps(pred)
    http_post_get.mobius_post(URI, AE_ID, AE_ID, send_data)

mqtt://203.250.148.120/sensor1RawDataSub?ct=json
sensor1RawDataSub
connected OK
subscribed: 1 (1,)
